'웹 로그 기반 조회수 예측 해커톤'  
데이터  
| 데이터 명              | 설명                           |
|-----------------------|--------------------------------|
| sessionID             | 세션 ID                        |
| userID                | 사용자 ID                      |
| TARGET                | 세션에서 발생한 총 조회수       |
| browser               | 사용된 브라우저                 |
| OS                    | 사용된 기기의 운영체제          |
| device                | 사용된 기기                    |
| new                   | 첫 방문 여부                   |
| quality               | 세션의 질                      |
| duration              | 총 세션 시간                   |
| bounced               | 이탈 여부                      |
| transaction           | 세션 내에서 발생한 거래의 수   |
| transaction_revenue   | 총 거래 수익                   |
| continent             | 세션이 발생한 대륙             |
| subcontinent          | 세션이 발생한 하위 대륙        |
| country               | 세션이 발생한 국가             |
| traffic_source        | 트래픽이 발생한 소스           |
| traffic_medium        | 트래픽 소스의 매체             |
| keyword               | 트래픽 소스의 키워드           |
| referral_path         | referral인 경우 설정되는 경로  |

이와같은 데이터를 통해서 TARGET(조회수)을 예측하는 것이 목표 


In [1]:
#데이터 불러오기
import pandas as pd
import numpy as np

train_data = pd.read_csv('./data/train.csv') #have TARGET
test_data = pd.read_csv('./data/test.csv')

y_data = train_data.pop('TARGET') #TARGET 데이터 

데이터 확인

In [2]:
train_data.head()

,sessionID,userID,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,0,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,groups.google.com,referral,NaN,Category6_Path_0000


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252289 entries, 0 to 252288
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   sessionID            252289 non-null  object 
 1   userID               252289 non-null  object 
 2   browser              252289 non-null  object 
 3   OS                   252289 non-null  object 
 4   device               252289 non-null  object 
 5   new                  252289 non-null  int64  
 6   quality              252289 non-null  float64
 7   duration             252289 non-null  float64
 8   bounced              252289 non-null  int64  
 9   transaction          252289 non-null  float64
 10  transaction_revenue  252289 non-null  float64
 11  continent            252289 non-null  object 
 12  subcontinent         252289 non-null  object 
 13  country              252289 non-null  object 
 14  traffic_source       252289 non-null  object 
 15  traffic_medium   

In [4]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79786 entries, 0 to 79785
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sessionID            79786 non-null  object 
 1   userID               79786 non-null  object 
 2   browser              79786 non-null  object 
 3   OS                   79786 non-null  object 
 4   device               79786 non-null  object 
 5   new                  79786 non-null  int64  
 6   quality              79786 non-null  float64
 7   duration             79786 non-null  float64
 8   bounced              79786 non-null  int64  
 9   transaction          79786 non-null  float64
 10  transaction_revenue  79786 non-null  float64
 11  continent            79786 non-null  object 
 12  subcontinent         79786 non-null  object 
 13  country              79786 non-null  object 
 14  traffic_source       79786 non-null  object 
 15  traffic_medium       79786 non-null 

In [5]:
y_data

0         17.0
1          3.0
2          1.0
3          1.0
4          1.0
          ... 
252284     1.0
252285     1.0
252286     5.0
252287     1.0
252288     6.0
Name: TARGET, Length: 252289, dtype: float64

train, test 데이터의 통합적인 전처리를 위한 total 데이터 생성

In [6]:
#전처리를 위한 데이터 한판
total_data = pd.concat([train_data, test_data]).reset_index(drop=True)
total_data

,sessionID,userID,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,0,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,groups.google.com,referral,NaN,Category6_Path_0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,Internet Explorer,Windows,tablet,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,search.xfinity.com,referral,NaN,Category1
332071,SESSION_332071,USER_269000,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
332072,SESSION_332072,USER_269001,Safari,Macintosh,desktop,1,1.0,22.0,0,0.0,0.0,Oceania,Australasia,Australia,google,organic,Category8,NaN
332073,SESSION_332073,USER_269002,Safari,iOS,mobile,1,1.0,135.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN


In [8]:
#결측치 확인
total_data.isnull().sum()

sessionID                   0
userID                      0
browser                     0
OS                          0
device                      0
new                         0
quality                     0
duration                    0
bounced                     0
transaction                 0
transaction_revenue         0
continent                   0
subcontinent                0
country                     0
traffic_source              0
traffic_medium              0
keyword                180745
referral_path          214998
dtype: int64

In [9]:
#결측치 비율 확인
total_data.isnull().mean() * 100

sessionID               0.000000
userID                  0.000000
browser                 0.000000
OS                      0.000000
device                  0.000000
new                     0.000000
quality                 0.000000
duration                0.000000
bounced                 0.000000
transaction             0.000000
transaction_revenue     0.000000
continent               0.000000
subcontinent            0.000000
country                 0.000000
traffic_source          0.000000
traffic_medium          0.000000
keyword                54.428969
referral_path          64.743808
dtype: float64

In [10]:
#열 종류 확인
for column in total_data.columns:
    unique_values = total_data[column].unique()
    print(f"{column} 열의 고유한 데이터 종류:")
    print(len(unique_values))

sessionID 열의 고유한 데이터 종류:
332075
userID 열의 고유한 데이터 종류:
269004
browser 열의 고유한 데이터 종류:
88
OS 열의 고유한 데이터 종류:
21
device 열의 고유한 데이터 종류:
3
new 열의 고유한 데이터 종류:
2
quality 열의 고유한 데이터 종류:
98
duration 열의 고유한 데이터 종류:
3194
bounced 열의 고유한 데이터 종류:
2
transaction 열의 고유한 데이터 종류:
8
transaction_revenue 열의 고유한 데이터 종류:
1505
continent 열의 고유한 데이터 종류:
6
subcontinent 열의 고유한 데이터 종류:
23
country 열의 고유한 데이터 종류:
207
traffic_source 열의 고유한 데이터 종류:
177
traffic_medium 열의 고유한 데이터 종류:
7
keyword 열의 고유한 데이터 종류:
760
referral_path 열의 고유한 데이터 종류:
1713


데이터 별 가설 수립 - 6개의 가설
- 1. 첫 방문 유무와 조회수의 관계
- 2. 세션의 질과 조회수의 관계
- 3. 세션 시간과 조회수의 관계
- 4. 이탈 여부와 조회수의 관계
- 5. 대륙별 조회수의 차이
- 6. 트래픽 소스 및 매체와 조회수의 관계

 가설1. 첫방문 유무와 조회수의 관계  
가설 : New(첫방문유무)가 TARGET(조회수)에 영향을 줄것이다

In [11]:
trd = 252289 #train 데이터 수
hypo1 = pd.DataFrame({ 'Y': y_data , 'new': total_data['new'][:trd]})
hypo1

,Y,new
0,17.0,0
1,3.0,1
2,1.0,1
3,1.0,1
4,1.0,0
...,...,...
252284,1.0,1
252285,1.0,0
252286,5.0,0
252287,1.0,1


In [12]:
hypo1.corr()

,Y,new
Y,1.000000,-0.066609
new,-0.066609,1.000000


가설1 기각.  
이유: 상관관계 값 |-0.066609| < 0.1

가설2. 세션의 질과 조회수의 관계  
가설: quality(세션의 질)이 높을수록 TARGET조회수가 높을 것이다

In [13]:
hypo2 = pd.DataFrame({ 'Y': y_data , 'quality': total_data['quality'][:trd]})
hypo2

,Y,quality
0,17.0,45.0
1,3.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
...,...,...
252284,1.0,1.0
252285,1.0,1.0
252286,5.0,2.0
252287,1.0,1.0


In [14]:
hypo2.corr()

,Y,quality
Y,1.000000,0.632723
quality,0.632723,1.000000


가설2 수용
이유: 상관관계 값|0.632723| > 0.1 

가설3. 세션 시간과 조회수의 관계  
가설: duration(세션의 총 시간)이 길수록 TARGET(조회수)가 높을 것이다.

In [15]:
hypo3 = pd.DataFrame({ 'Y': y_data , 'duration': total_data['duration'][:trd]})
hypo3

,Y,duration
0,17.0,839.0
1,3.0,39.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
...,...,...
252284,1.0,0.0
252285,1.0,0.0
252286,5.0,69.0
252287,1.0,28.0


In [16]:
hypo3.corr()

,Y,duration
Y,1.00000,0.59534
duration,0.59534,1.00000


가설3 수용  
이유: 상관관계 값|0.59534| > 0.1 

가설4. 이탈 여부와 조회수의 관계  
가설: bounced(이탈 여부)가 TARGET(조회수)에 영향을 줄 것이다

In [17]:
hypo4 = pd.DataFrame({ 'Y': y_data , 'bounced': total_data['bounced'][:trd]})
hypo4

,Y,bounced
0,17.0,0
1,3.0,0
2,1.0,1
3,1.0,1
4,1.0,1
...,...,...
252284,1.0,1
252285,1.0,1
252286,5.0,0
252287,1.0,0


In [18]:
hypo4.corr()

,Y,bounced
Y,1.000000,-0.464659
bounced,-0.464659,1.000000


가설4 수용  
이유: 상관관계 값|-0.464659| > 0.1 

가설5. 대륙별 조회수의 차이  
가설: continent,subcontinent, country (대륙, 하위대륙, 국가)에 따라 TARGET(조회수)가 다를 것이다

In [19]:
hypo5 = pd.DataFrame({ 'Y': y_data , 'continent': total_data['continent'][:trd],  'subcontinent': total_data['subcontinent'][:trd],  'country': total_data['country'][:trd]})
hypo5

,Y,continent,subcontinent,country
0,17.0,Americas,Northern America,United States
1,3.0,Europe,Western Europe,Germany
2,1.0,Asia,Southeast Asia,Malaysia
3,1.0,Americas,Northern America,United States
4,1.0,Americas,Northern America,United States
...,...,...,...,...
252284,1.0,Europe,Northern Europe,United Kingdom
252285,1.0,Americas,Northern America,United States
252286,5.0,Americas,Northern America,United States
252287,1.0,Africa,Northern Africa,Egypt


In [20]:
#문자열 데이터 원핫인코딩
hypo5 = pd.get_dummies(hypo5)
hypo5

,Y,continent_(not set),continent_Africa,continent_Americas,continent_Asia,continent_Europe,continent_Oceania,subcontinent_(not set),subcontinent_Australasia,subcontinent_Caribbean,...,country_United Kingdom,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Vietnam,country_Yemen,country_Zambia,country_Zimbabwe
0,17.0,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,3.0,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1.0,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1.0,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,1.0,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252284,1.0,False,False,False,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
252285,1.0,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
252286,5.0,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
252287,1.0,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
hypo5_corr = hypo5.corr()

In [22]:
hypo5_corr.pop("Y")
hypo5_corr

,continent_(not set),continent_Africa,continent_Americas,continent_Asia,continent_Europe,continent_Oceania,subcontinent_(not set),subcontinent_Australasia,subcontinent_Caribbean,subcontinent_Central America,...,country_United Kingdom,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Vietnam,country_Yemen,country_Zambia,country_Zimbabwe
Y,-0.002251,-0.049874,0.153919,-0.081872,-0.077253,-0.007962,-0.002251,-0.007967,-0.003522,-0.012929,...,-0.041168,0.159252,0.002306,-0.002429,-0.000576,0.001227,-0.030581,-0.000296,-0.002121,-0.002276
continent_(not set),1.000000,-0.007615,-0.036623,-0.019786,-0.019001,-0.004594,1.000000,-0.004573,-0.001736,-0.004752,...,-0.007426,-0.030633,-0.000682,-0.000436,-0.000073,-0.001141,-0.004175,-0.000454,-0.000291,-0.000262
continent_Africa,-0.007615,1.000000,-0.209133,-0.112986,-0.108501,-0.026236,-0.007615,-0.026113,-0.009915,-0.027136,...,-0.042407,-0.174926,-0.003895,-0.002491,-0.000415,-0.006515,-0.023841,-0.002593,0.038190,0.034424
continent_Americas,-0.036623,-0.209133,1.000000,-0.543364,-0.521796,-0.126170,-0.036623,-0.125582,0.047408,0.129756,...,-0.203942,0.836434,0.018626,-0.011981,-0.001997,0.031152,-0.114654,-0.012470,-0.007987,-0.007199
continent_Asia,-0.019786,-0.112986,-0.543364,1.000000,-0.281905,-0.068165,-0.019786,-0.067847,-0.025760,-0.070505,...,-0.110182,-0.454488,-0.010121,0.022049,-0.001079,-0.016927,0.211008,0.022949,-0.004315,-0.003889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
country_Venezuela,-0.001141,-0.006515,0.031152,-0.016927,-0.016255,-0.003930,-0.001141,-0.003912,-0.001485,-0.004065,...,-0.006353,-0.026206,-0.000584,-0.000373,-0.000062,1.000000,-0.003572,-0.000388,-0.000249,-0.000224
country_Vietnam,-0.004175,-0.023841,-0.114654,0.211008,-0.059484,-0.014383,-0.004175,-0.014316,-0.005436,-0.014877,...,-0.023249,-0.095901,-0.002136,-0.001366,-0.000228,-0.003572,1.000000,-0.001422,-0.000910,-0.000821
country_Yemen,-0.000454,-0.002593,-0.012470,0.022949,-0.006470,-0.001564,-0.000454,-0.001557,-0.000591,-0.001618,...,-0.002529,-0.010430,-0.000232,-0.000149,-0.000025,-0.000388,-0.001422,1.000000,-0.000099,-0.000089
country_Zambia,-0.000291,0.038190,-0.007987,-0.004315,-0.004144,-0.001002,-0.000291,-0.000997,-0.000379,-0.001036,...,-0.001620,-0.006680,-0.000149,-0.000095,-0.000016,-0.000249,-0.000910,-0.000099,1.000000,-0.000057


In [23]:
lst_5 = []
for lst in hypo5_corr[:1]:
    if abs(hypo5_corr[lst][0]) > 0.1:
        lst_5.append(lst)
    
lst_5

C:\Users\user\AppData\Local\Temp\ipykernel_22428\1122493518.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(hypo5_corr[lst][0]) > 0.1:


['continent_Americas',
 'subcontinent_Northern America',
 'country_United States']

In [24]:
hypo5_corr[lst_5][:1]

,continent_Americas,subcontinent_Northern America,country_United States
Y,0.153919,0.167332,0.159252


가설5 부분 수용  
상관관계가 0.1 이 넘는 요소들 lst_5 에 저장, 추후 total_data에서 열 교체

가설6. 트래픽 소스 및 매체와 조회수의 관계  
가설: traffic_source(트래픽 소스)와 traffic_medium(매체)에 따라 TARGET(조회수)가 다를 것이다.

In [25]:
hypo6 = pd.DataFrame({ 'Y': y_data , 'traffic_source': total_data['traffic_source'][:trd],  'traffic_medium': total_data['traffic_medium'][:trd]})
hypo6

,Y,traffic_source,traffic_medium
0,17.0,google,organic
1,3.0,google,organic
2,1.0,(direct),(none)
3,1.0,Partners,affiliate
4,1.0,groups.google.com,referral
...,...,...,...
252284,1.0,youtube.com,referral
252285,1.0,google,organic
252286,5.0,(direct),(none)
252287,1.0,youtube.com,referral


In [27]:
#문자열 데이터 원핫인코딩
hypo6 = pd.get_dummies(hypo6)
hypo6

,Y,traffic_source_(direct),traffic_source_Partners,traffic_source_adwords.google.com,traffic_source_amp.reddit.com,traffic_source_analytics.google.com,traffic_source_arstechnica.com,traffic_source_ask,traffic_source_au.search.yahoo.com,traffic_source_aulagrado.esic.edu,...,traffic_source_yandex,traffic_source_youtube.com,traffic_source_youtube.thinkwithgoogle.com,traffic_medium_(none),traffic_medium_(not set),traffic_medium_affiliate,traffic_medium_cpc,traffic_medium_cpm,traffic_medium_organic,traffic_medium_referral
0,17.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,3.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,1.0,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,1.0,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252284,1.0,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
252285,1.0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
252286,5.0,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
252287,1.0,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True


In [28]:
hypo6_corr = hypo6.corr()

In [29]:
hypo6_corr.pop("Y")
hypo6_corr

,traffic_source_(direct),traffic_source_Partners,traffic_source_adwords.google.com,traffic_source_amp.reddit.com,traffic_source_analytics.google.com,traffic_source_arstechnica.com,traffic_source_ask,traffic_source_au.search.yahoo.com,traffic_source_aulagrado.esic.edu,traffic_source_away.vk.com,...,traffic_source_yandex,traffic_source_youtube.com,traffic_source_youtube.thinkwithgoogle.com,traffic_medium_(none),traffic_medium_(not set),traffic_medium_affiliate,traffic_medium_cpc,traffic_medium_cpm,traffic_medium_organic,traffic_medium_referral
Y,0.099120,-0.012860,-0.002815,0.000464,-0.018047,-0.002394,-0.002155,-0.000788,-0.003754,-0.008147,...,-0.000937,-0.126561,0.000093,0.099140,-0.001852,-0.012823,-0.025083,-0.004410,0.039063,-0.121338
traffic_source_(direct),1.000000,-0.081492,-0.006126,-0.001556,-0.097457,-0.003112,-0.005502,-0.001906,-0.006602,-0.021436,...,-0.001100,-0.284560,-0.003812,0.999956,-0.002695,-0.081461,-0.112146,-0.023811,-0.475692,-0.342624
traffic_source_Partners,-0.081492,1.000000,-0.001635,-0.000415,-0.026004,-0.000830,-0.001468,-0.000508,-0.001762,-0.005720,...,-0.000294,-0.075927,-0.001017,-0.081489,0.021808,0.999619,-0.029923,-0.006578,-0.126925,-0.091420
traffic_source_adwords.google.com,-0.006126,-0.001635,1.000000,-0.000031,-0.001955,-0.000062,-0.000110,-0.000038,-0.000132,-0.000430,...,-0.000022,-0.005708,-0.000076,-0.006126,-0.000054,-0.001634,-0.002250,-0.000494,-0.009542,0.017881
traffic_source_amp.reddit.com,-0.001556,-0.000415,-0.000031,1.000000,-0.000497,-0.000016,-0.000028,-0.000010,-0.000034,-0.000109,...,-0.000006,-0.001450,-0.000019,-0.001556,-0.000014,-0.000415,-0.000571,-0.000126,-0.002424,0.004541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
traffic_medium_affiliate,-0.081461,0.999619,-0.001634,-0.000415,-0.025994,-0.000830,-0.001467,-0.000508,-0.001761,-0.005717,...,-0.000293,-0.075898,-0.001017,-0.081458,-0.000719,1.000000,-0.029912,-0.006575,-0.126877,-0.091385
traffic_medium_cpc,-0.112146,-0.029923,-0.002250,-0.000571,-0.035785,-0.001143,-0.002020,-0.000700,-0.002424,-0.007871,...,-0.000404,-0.104487,-0.001400,-0.112141,-0.000990,-0.029912,1.000000,-0.009052,-0.174668,-0.125807
traffic_medium_cpm,-0.023811,-0.006578,-0.000494,-0.000126,-0.007866,-0.000251,-0.000444,-0.000154,-0.000533,-0.001730,...,-0.000089,-0.022968,-0.000308,-0.024651,-0.000218,-0.006575,-0.009052,1.000000,-0.038395,-0.027655
traffic_medium_organic,-0.475692,-0.126925,-0.009542,-0.002424,-0.151790,-0.004847,0.011565,-0.002968,-0.010283,-0.033387,...,0.002313,-0.443206,-0.005937,-0.475671,-0.004198,-0.126877,-0.174668,-0.038395,1.000000,-0.533641


In [30]:
lst_6 = []
for lst in hypo6_corr[:1]:
    if abs(hypo6_corr[lst][0]) > 0.1:
        lst_6.append(lst)
    
lst_6

C:\Users\user\AppData\Local\Temp\ipykernel_22428\3272213306.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(hypo6_corr[lst][0]) > 0.1:


['traffic_source_youtube.com', 'traffic_medium_referral']

가설6 부분 수용  
상관관계가 0.1 이 넘는 요소들 lst_6 에 저장, 추후 total_data에서 열 교체

가설정리  
- 가설1. 첫 방문 유무와 조회수의 관계: False  
- 가설2. 세션의 질과 조회수의 관계: True  
- 가설3. 세션 시간과 조회수의 관계: True  
- 가설4. 이탈 여부와 조회수의 관계: True  
- 가설5. 대륙별 조회수의 차이: True - > lst_5  
- 가설6. 트래픽 소스 및 매체와 조회수의 관계: True -> lst_6  

가설에 따른 total_data 정리

In [31]:
total_data

,sessionID,userID,browser,OS,device,new,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,0,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,0,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,groups.google.com,referral,NaN,Category6_Path_0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,Internet Explorer,Windows,tablet,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,search.xfinity.com,referral,NaN,Category1
332071,SESSION_332071,USER_269000,Chrome,Macintosh,desktop,1,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
332072,SESSION_332072,USER_269001,Safari,Macintosh,desktop,1,1.0,22.0,0,0.0,0.0,Oceania,Australasia,Australia,google,organic,Category8,NaN
332073,SESSION_332073,USER_269002,Safari,iOS,mobile,1,1.0,135.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN


In [32]:
#가설 1, new 열 제거
total_data.pop("new")
total_data

,sessionID,userID,browser,OS,device,quality,duration,bounced,transaction,transaction_revenue,continent,subcontinent,country,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,45.0,839.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1.0,39.0,0,0.0,0.0,Europe,Western Europe,Germany,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1.0,0.0,1,0.0,0.0,Asia,Southeast Asia,Malaysia,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,groups.google.com,referral,NaN,Category6_Path_0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,Internet Explorer,Windows,tablet,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,search.xfinity.com,referral,NaN,Category1
332071,SESSION_332071,USER_269000,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN
332072,SESSION_332072,USER_269001,Safari,Macintosh,desktop,1.0,22.0,0,0.0,0.0,Oceania,Australasia,Australia,google,organic,Category8,NaN
332073,SESSION_332073,USER_269002,Safari,iOS,mobile,1.0,135.0,0,0.0,0.0,Americas,Northern America,United States,google,organic,Category8,NaN


In [33]:
#가설 5, total 데이터에서 lst_5 값 불러오기 위한 준비
hypo5_input = pd.DataFrame({'continent': total_data['continent'],  'subcontinent': total_data['subcontinent'],  'country': total_data['country']})

In [34]:
hypo5_gdis = pd.get_dummies(hypo5_input)
hypo5_gdis

,continent_(not set),continent_Africa,continent_Americas,continent_Asia,continent_Europe,continent_Oceania,subcontinent_(not set),subcontinent_Australasia,subcontinent_Caribbean,subcontinent_Central America,...,country_United Kingdom,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Vietnam,country_Yemen,country_Zambia,country_Zimbabwe
0,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
332071,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
332072,False,False,False,False,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
332073,False,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [35]:
#가설 5 진행, continent,subcontinent, country (대륙, 하위대륙, 국가) 열 제거 및 lst_5 열 추가
total_data = total_data.drop(columns=["continent","subcontinent","country"])
total_data

,sessionID,userID,browser,OS,device,quality,duration,bounced,transaction,transaction_revenue,traffic_source,traffic_medium,keyword,referral_path
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,45.0,839.0,0,0.0,0.0,google,organic,Category8,NaN
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1.0,39.0,0,0.0,0.0,google,organic,Category8,NaN
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1.0,0.0,1,0.0,0.0,(direct),(none),NaN,NaN
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,Partners,affiliate,NaN,NaN
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,1.0,0.0,1,0.0,0.0,groups.google.com,referral,NaN,Category6_Path_0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,Internet Explorer,Windows,tablet,1.0,0.0,1,0.0,0.0,search.xfinity.com,referral,NaN,Category1
332071,SESSION_332071,USER_269000,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,google,organic,Category8,NaN
332072,SESSION_332072,USER_269001,Safari,Macintosh,desktop,1.0,22.0,0,0.0,0.0,google,organic,Category8,NaN
332073,SESSION_332073,USER_269002,Safari,iOS,mobile,1.0,135.0,0,0.0,0.0,google,organic,Category8,NaN


In [36]:
for lst in lst_5:
    total_data[lst] = hypo5_gdis[lst]

total_data

,sessionID,userID,browser,OS,device,quality,duration,bounced,transaction,transaction_revenue,traffic_source,traffic_medium,keyword,referral_path,continent_Americas,subcontinent_Northern America,country_United States
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,45.0,839.0,0,0.0,0.0,google,organic,Category8,NaN,True,True,True
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1.0,39.0,0,0.0,0.0,google,organic,Category8,NaN,False,False,False
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1.0,0.0,1,0.0,0.0,(direct),(none),NaN,NaN,False,False,False
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,Partners,affiliate,NaN,NaN,True,True,True
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,1.0,0.0,1,0.0,0.0,groups.google.com,referral,NaN,Category6_Path_0000,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,Internet Explorer,Windows,tablet,1.0,0.0,1,0.0,0.0,search.xfinity.com,referral,NaN,Category1,True,True,True
332071,SESSION_332071,USER_269000,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,google,organic,Category8,NaN,True,True,True
332072,SESSION_332072,USER_269001,Safari,Macintosh,desktop,1.0,22.0,0,0.0,0.0,google,organic,Category8,NaN,False,False,False
332073,SESSION_332073,USER_269002,Safari,iOS,mobile,1.0,135.0,0,0.0,0.0,google,organic,Category8,NaN,True,True,True


In [37]:
#가설 6, total 데이터에서 lst_6 값 불러오기 위한 준비
hypo6_input = pd.DataFrame({'traffic_source': total_data['traffic_source'],  'traffic_medium': total_data['traffic_medium']})
hypo6_input

,traffic_source,traffic_medium
0,google,organic
1,google,organic
2,(direct),(none)
3,Partners,affiliate
4,groups.google.com,referral
...,...,...
332070,search.xfinity.com,referral
332071,google,organic
332072,google,organic
332073,google,organic


In [38]:
hypo6_gdis = pd.get_dummies(hypo6_input)
hypo6_gdis

,traffic_source_(direct),traffic_source_Partners,traffic_source_adwords.google.com,traffic_source_amp.reddit.com,traffic_source_analytics.google.com,traffic_source_arstechnica.com,traffic_source_ask,traffic_source_au.search.yahoo.com,traffic_source_aulagrado.esic.edu,traffic_source_away.vk.com,...,traffic_source_yandex,traffic_source_youtube.com,traffic_source_youtube.thinkwithgoogle.com,traffic_medium_(none),traffic_medium_(not set),traffic_medium_affiliate,traffic_medium_cpc,traffic_medium_cpm,traffic_medium_organic,traffic_medium_referral
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
332071,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
332072,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
332073,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [39]:
#가설 6 진행, traffic_source(트래픽 소스)와 traffic_medium(매체) 열 제거 및 lst_6 열 추가
total_data = total_data.drop(columns=["traffic_source","traffic_medium"])
total_data

,sessionID,userID,browser,OS,device,quality,duration,bounced,transaction,transaction_revenue,keyword,referral_path,continent_Americas,subcontinent_Northern America,country_United States
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,45.0,839.0,0,0.0,0.0,Category8,NaN,True,True,True
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1.0,39.0,0,0.0,0.0,Category8,NaN,False,False,False
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1.0,0.0,1,0.0,0.0,NaN,NaN,False,False,False
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,NaN,NaN,True,True,True
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,1.0,0.0,1,0.0,0.0,NaN,Category6_Path_0000,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,Internet Explorer,Windows,tablet,1.0,0.0,1,0.0,0.0,NaN,Category1,True,True,True
332071,SESSION_332071,USER_269000,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,Category8,NaN,True,True,True
332072,SESSION_332072,USER_269001,Safari,Macintosh,desktop,1.0,22.0,0,0.0,0.0,Category8,NaN,False,False,False
332073,SESSION_332073,USER_269002,Safari,iOS,mobile,1.0,135.0,0,0.0,0.0,Category8,NaN,True,True,True


In [40]:
for lst in lst_6:
    total_data[lst] = hypo6_gdis[lst]

total_data

,sessionID,userID,browser,OS,device,quality,duration,bounced,transaction,transaction_revenue,keyword,referral_path,continent_Americas,subcontinent_Northern America,country_United States,traffic_source_youtube.com,traffic_medium_referral
0,SESSION_000000,USER_000000,Chrome,Macintosh,desktop,45.0,839.0,0,0.0,0.0,Category8,NaN,True,True,True,False,False
1,SESSION_000001,USER_000001,Chrome,Windows,desktop,1.0,39.0,0,0.0,0.0,Category8,NaN,False,False,False,False,False
2,SESSION_000002,USER_000002,Samsung Internet,Android,mobile,1.0,0.0,1,0.0,0.0,NaN,NaN,False,False,False,False,False
3,SESSION_000003,USER_000003,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,NaN,NaN,True,True,True,False,False
4,SESSION_000004,USER_000004,Chrome,iOS,mobile,1.0,0.0,1,0.0,0.0,NaN,Category6_Path_0000,True,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,Internet Explorer,Windows,tablet,1.0,0.0,1,0.0,0.0,NaN,Category1,True,True,True,False,True
332071,SESSION_332071,USER_269000,Chrome,Macintosh,desktop,1.0,0.0,1,0.0,0.0,Category8,NaN,True,True,True,False,False
332072,SESSION_332072,USER_269001,Safari,Macintosh,desktop,1.0,22.0,0,0.0,0.0,Category8,NaN,False,False,False,False,False
332073,SESSION_332073,USER_269002,Safari,iOS,mobile,1.0,135.0,0,0.0,0.0,Category8,NaN,True,True,True,False,False


나머지 값 원핫인코딩(get_dummies) 사용해서 상관관계 확인 후 열 제거 및 추가 (browser, OS, device)   
keyword, referral_path 결측치 각각 54%, 64% 이므로 데이터가 정상적인 역할을 못할것이라 판단 - 열 제거 가설 0

In [41]:
hypo0_input = pd.DataFrame({'browser': total_data['browser'],  'OS': total_data['OS'],'device': total_data['device']})
hypo0_input

,browser,OS,device
0,Chrome,Macintosh,desktop
1,Chrome,Windows,desktop
2,Samsung Internet,Android,mobile
3,Chrome,Macintosh,desktop
4,Chrome,iOS,mobile
...,...,...,...
332070,Internet Explorer,Windows,tablet
332071,Chrome,Macintosh,desktop
332072,Safari,Macintosh,desktop
332073,Safari,iOS,mobile


In [42]:
hypo0_gdis = pd.get_dummies(hypo0_input)
hypo0_gdis

,browser_(not set),browser_+Simple Browser,browser_;__CT_JOB_ID__:0a075729-93a5-43d0-9638-4cbd41d5f5a5;,browser_;__CT_JOB_ID__:0b39e7ca-1431-42e3-ba1f-9d8951a65840;,browser_;__CT_JOB_ID__:2547db0b-ec43-452a-a0d4-ff42b7dc7907;,browser_;__CT_JOB_ID__:2e0eca60-83ab-482d-bb81-343d113254fb;,browser_;__CT_JOB_ID__:4333777f-bb0c-4a18-935e-df5658dbce2d;,browser_;__CT_JOB_ID__:58e2ecba-7666-4a10-b498-8216457ce472;,browser_;__CT_JOB_ID__:65da7e5f-0f05-4b5d-8d31-1f4d470a2b82;,browser_;__CT_JOB_ID__:6e9dcf2f-f58f-4938-91e3-77e00868177b;,...,OS_SunOS,OS_SymbianOS,OS_Tizen,OS_Windows,OS_Windows Phone,OS_Xbox,OS_iOS,device_desktop,device_mobile,device_tablet
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
332071,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
332072,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
332073,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [43]:
hypo0_gdis['Y'] = y_data
hypo0_gdis

,browser_(not set),browser_+Simple Browser,browser_;__CT_JOB_ID__:0a075729-93a5-43d0-9638-4cbd41d5f5a5;,browser_;__CT_JOB_ID__:0b39e7ca-1431-42e3-ba1f-9d8951a65840;,browser_;__CT_JOB_ID__:2547db0b-ec43-452a-a0d4-ff42b7dc7907;,browser_;__CT_JOB_ID__:2e0eca60-83ab-482d-bb81-343d113254fb;,browser_;__CT_JOB_ID__:4333777f-bb0c-4a18-935e-df5658dbce2d;,browser_;__CT_JOB_ID__:58e2ecba-7666-4a10-b498-8216457ce472;,browser_;__CT_JOB_ID__:65da7e5f-0f05-4b5d-8d31-1f4d470a2b82;,browser_;__CT_JOB_ID__:6e9dcf2f-f58f-4938-91e3-77e00868177b;,...,OS_SymbianOS,OS_Tizen,OS_Windows,OS_Windows Phone,OS_Xbox,OS_iOS,device_desktop,device_mobile,device_tablet,Y
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,17.0
1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,3.0
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1.0
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,1.0
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332070,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,NaN
332071,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,NaN
332072,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,NaN
332073,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,NaN


In [44]:
hypo0_gdis[:trd]

,browser_(not set),browser_+Simple Browser,browser_;__CT_JOB_ID__:0a075729-93a5-43d0-9638-4cbd41d5f5a5;,browser_;__CT_JOB_ID__:0b39e7ca-1431-42e3-ba1f-9d8951a65840;,browser_;__CT_JOB_ID__:2547db0b-ec43-452a-a0d4-ff42b7dc7907;,browser_;__CT_JOB_ID__:2e0eca60-83ab-482d-bb81-343d113254fb;,browser_;__CT_JOB_ID__:4333777f-bb0c-4a18-935e-df5658dbce2d;,browser_;__CT_JOB_ID__:58e2ecba-7666-4a10-b498-8216457ce472;,browser_;__CT_JOB_ID__:65da7e5f-0f05-4b5d-8d31-1f4d470a2b82;,browser_;__CT_JOB_ID__:6e9dcf2f-f58f-4938-91e3-77e00868177b;,...,OS_SymbianOS,OS_Tizen,OS_Windows,OS_Windows Phone,OS_Xbox,OS_iOS,device_desktop,device_mobile,device_tablet,Y
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,17.0
1,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,False,3.0
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1.0
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,1.0
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252284,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1.0
252285,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,1.0
252286,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,5.0
252287,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,1.0


In [45]:
hypo0_corr = hypo0_gdis[:trd].corr()

In [46]:
hypo0_corr

,browser_(not set),browser_+Simple Browser,browser_;__CT_JOB_ID__:0a075729-93a5-43d0-9638-4cbd41d5f5a5;,browser_;__CT_JOB_ID__:0b39e7ca-1431-42e3-ba1f-9d8951a65840;,browser_;__CT_JOB_ID__:2547db0b-ec43-452a-a0d4-ff42b7dc7907;,browser_;__CT_JOB_ID__:2e0eca60-83ab-482d-bb81-343d113254fb;,browser_;__CT_JOB_ID__:4333777f-bb0c-4a18-935e-df5658dbce2d;,browser_;__CT_JOB_ID__:58e2ecba-7666-4a10-b498-8216457ce472;,browser_;__CT_JOB_ID__:65da7e5f-0f05-4b5d-8d31-1f4d470a2b82;,browser_;__CT_JOB_ID__:6e9dcf2f-f58f-4938-91e3-77e00868177b;,...,OS_SymbianOS,OS_Tizen,OS_Windows,OS_Windows Phone,OS_Xbox,OS_iOS,device_desktop,device_mobile,device_tablet,Y
browser_(not set),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
browser_+Simple Browser,NaN,1.000000,-0.000007,-0.000007,-0.000007,-0.000007,-0.000007,-0.000007,NaN,-0.000007,...,-0.000007,-0.000123,-0.002526,-0.000078,-0.000038,-0.001261,0.002461,-0.002274,-0.000652,0.003176
browser_;__CT_JOB_ID__:0a075729-93a5-43d0-9638-4cbd41d5f5a5;,NaN,-0.000007,1.000000,-0.000004,-0.000004,-0.000004,-0.000004,-0.000004,NaN,-0.000004,...,-0.000004,-0.000071,-0.001459,-0.000045,-0.000022,-0.000728,0.001421,-0.001313,-0.000376,-0.000937
browser_;__CT_JOB_ID__:0b39e7ca-1431-42e3-ba1f-9d8951a65840;,NaN,-0.000007,-0.000004,1.000000,-0.000004,-0.000004,-0.000004,-0.000004,NaN,-0.000004,...,-0.000004,-0.000071,-0.001459,-0.000045,-0.000022,-0.000728,0.001421,-0.001313,-0.000376,-0.000937
browser_;__CT_JOB_ID__:2547db0b-ec43-452a-a0d4-ff42b7dc7907;,NaN,-0.000007,-0.000004,-0.000004,1.000000,-0.000004,-0.000004,-0.000004,NaN,-0.000004,...,-0.000004,-0.000071,-0.001459,-0.000045,-0.000022,-0.000728,0.001421,-0.001313,-0.000376,-0.000937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OS_iOS,NaN,-0.001261,-0.000728,-0.000728,-0.000728,-0.000728,-0.000728,-0.000728,NaN,-0.000728,...,-0.000728,-0.013057,-0.268015,-0.008242,-0.004055,1.000000,-0.512320,0.422971,0.262403,-0.032712
device_desktop,NaN,0.002461,0.001421,0.001421,0.001421,0.001421,0.001421,0.001421,NaN,0.001421,...,-0.002789,-0.049769,0.516977,-0.031564,0.007913,-0.512320,1.000000,-0.923837,-0.264698,0.072923
device_mobile,NaN,-0.002274,-0.001313,-0.001313,-0.001313,-0.001313,-0.001313,-0.001313,NaN,-0.001313,...,0.003019,0.052675,-0.481967,0.034166,-0.007310,0.422971,-0.923837,1.000000,-0.124595,-0.074464
device_tablet,NaN,-0.000652,-0.000376,-0.000376,-0.000376,-0.000376,-0.000376,-0.000376,NaN,-0.000376,...,-0.000376,-0.003696,-0.125847,-0.004257,-0.002095,0.262403,-0.264698,-0.124595,1.000000,-0.001429


In [47]:
lst_0 = []
for lst in hypo0_corr[:0]:
    if abs(hypo0_corr[lst][112]) > 0.1:
        lst_0.append(lst)
    
lst_0

C:\Users\user\AppData\Local\Temp\ipykernel_22428\1743349155.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(hypo0_corr[lst][112]) > 0.1:


['Y']

In [48]:
for lst in hypo0_corr[:0]:
    print(hypo0_corr[lst][112])

nan
0.0031758459672098223
-0.0009368778881793545
-0.0009368778881793479
-0.0009368778881793481
-0.0009368778881793481
-0.0009368778881793477
-0.0009368778881793476
nan
-0.0009368778881793519
-0.000936877888179351
-0.000936877888179349
-0.000936877888179348
-0.0009368778881793466
-0.0009368778881793542
-0.0009368778881793477
-0.0009368778881793477
-0.0009368778881793483
-0.0009368778881793468
-0.0009368778881793477
-0.000936877888179348
-0.0009368778881793479
-0.00424676817248007
0.0019968432795681433
-0.006313056331508139
-0.05531712969583294
-0.0013249480416174947
-0.0028769100047348365
-0.00120545936077368
0.09558087760089208
-0.002042230733302827
-0.0020949392521732767
-0.016423311206031486
-0.015360609307252925
-0.027541419218361837
-0.001873766917034183
-0.0016227265348673168
-0.0009368778881793476
-0.002882165174015942
-0.001581935104686099
-0.0016227265348672732
-0.002644960085149557
-0.0009368778881793512
0.001482155587972266
-0.0020750994468996826
-0.0009368778881793545
-0.006

C:\Users\user\AppData\Local\Temp\ipykernel_22428\3260788046.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(hypo0_corr[lst][112])


browser, OS, device 의 TARGET과의 상관관계를 확인했을때 유의한 수치를 가진 값이 없다고 확인, 열 제거  
과다 결측 값들도 제거

In [49]:
total_data = total_data.drop(columns=["browser","OS","device"])

In [50]:
total_data = total_data.drop(columns=["keyword","referral_path"])
total_data

,sessionID,userID,quality,duration,bounced,transaction,transaction_revenue,continent_Americas,subcontinent_Northern America,country_United States,traffic_source_youtube.com,traffic_medium_referral
0,SESSION_000000,USER_000000,45.0,839.0,0,0.0,0.0,True,True,True,False,False
1,SESSION_000001,USER_000001,1.0,39.0,0,0.0,0.0,False,False,False,False,False
2,SESSION_000002,USER_000002,1.0,0.0,1,0.0,0.0,False,False,False,False,False
3,SESSION_000003,USER_000003,1.0,0.0,1,0.0,0.0,True,True,True,False,False
4,SESSION_000004,USER_000004,1.0,0.0,1,0.0,0.0,True,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,1.0,0.0,1,0.0,0.0,True,True,True,False,True
332071,SESSION_332071,USER_269000,1.0,0.0,1,0.0,0.0,True,True,True,False,False
332072,SESSION_332072,USER_269001,1.0,22.0,0,0.0,0.0,False,False,False,False,False
332073,SESSION_332073,USER_269002,1.0,135.0,0,0.0,0.0,True,True,True,False,False


In [51]:
#결측값 확인
total_data.isnull().sum()

sessionID                        0
userID                           0
quality                          0
duration                         0
bounced                          0
transaction                      0
transaction_revenue              0
continent_Americas               0
subcontinent_Northern America    0
country_United States            0
traffic_source_youtube.com       0
traffic_medium_referral          0
dtype: int64

정규화 과정을 거쳐야함, float, int 값을 정규화 (quality, duration, transaction, transaction_revenue)

In [52]:
#정규화
# 정규화할 열 선택
columns_to_normalize = ['quality','duration','transaction','transaction_revenue']

for column in columns_to_normalize:
    total_data[column] = (total_data[column] - total_data[column].min()) / (total_data[column].max() - total_data[column].min())
    
total_data

,sessionID,userID,quality,duration,bounced,transaction,transaction_revenue,continent_Americas,subcontinent_Northern America,country_United States,traffic_source_youtube.com,traffic_medium_referral
0,SESSION_000000,USER_000000,0.453608,0.075004,0,0.0,0.0,True,True,True,False,False
1,SESSION_000001,USER_000001,0.000000,0.003487,0,0.0,0.0,False,False,False,False,False
2,SESSION_000002,USER_000002,0.000000,0.000000,1,0.0,0.0,False,False,False,False,False
3,SESSION_000003,USER_000003,0.000000,0.000000,1,0.0,0.0,True,True,True,False,False
4,SESSION_000004,USER_000004,0.000000,0.000000,1,0.0,0.0,True,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
332070,SESSION_332070,USER_268999,0.000000,0.000000,1,0.0,0.0,True,True,True,False,True
332071,SESSION_332071,USER_269000,0.000000,0.000000,1,0.0,0.0,True,True,True,False,False
332072,SESSION_332072,USER_269001,0.000000,0.001967,0,0.0,0.0,False,False,False,False,False
332073,SESSION_332073,USER_269002,0.000000,0.012069,0,0.0,0.0,True,True,True,False,False


In [53]:
# ID들 빼내기
session_id = total_data.pop("sessionID")
user_id = total_data.pop("userID")

In [54]:
total_data

,quality,duration,bounced,transaction,transaction_revenue,continent_Americas,subcontinent_Northern America,country_United States,traffic_source_youtube.com,traffic_medium_referral
0,0.453608,0.075004,0,0.0,0.0,True,True,True,False,False
1,0.000000,0.003487,0,0.0,0.0,False,False,False,False,False
2,0.000000,0.000000,1,0.0,0.0,False,False,False,False,False
3,0.000000,0.000000,1,0.0,0.0,True,True,True,False,False
4,0.000000,0.000000,1,0.0,0.0,True,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...
332070,0.000000,0.000000,1,0.0,0.0,True,True,True,False,True
332071,0.000000,0.000000,1,0.0,0.0,True,True,True,False,False
332072,0.000000,0.001967,0,0.0,0.0,False,False,False,False,False
332073,0.000000,0.012069,0,0.0,0.0,True,True,True,False,False


total 데이터 전처리 완료

In [55]:
#기존 데이터 형태로 전환
train_data_aft = total_data[:trd]
test_data_aft = total_data[trd:]

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# 학습 데이터와 테스트 데이터로 분할
X_train, X_val, y_train, y_val = train_test_split(train_data_aft, y_data, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((201831, 10), (50458, 10), (201831,), (50458,))

In [57]:
from sklearn.metrics import r2_score
# 랜덤 포레스트 회귀 모델 생성 및 학습
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(X_train, y_train)

r2_score(rf_regressor.predict(X_val), y_val)

0.6030455390673986

In [58]:
target = rf_regressor.predict(test_data_aft)
test_sessionID = session_id[len(y_data):].reset_index(drop=True)

In [59]:
target.shape

(79786,)

In [60]:
test_sessionID.shape

(79786,)

In [61]:
test_sessionID

0        SESSION_252289
1        SESSION_252290
2        SESSION_252291
3        SESSION_252292
4        SESSION_252293
              ...      
79781    SESSION_332070
79782    SESSION_332071
79783    SESSION_332072
79784    SESSION_332073
79785    SESSION_332074
Name: sessionID, Length: 79786, dtype: object

In [62]:
target

array([22.44      ,  1.        ,  2.75258047, ...,  2.56932496,
        4.44960371,  1.        ])

submission csv 파일로 만들기

In [63]:
df_submission = pd.DataFrame([test_sessionID, target]).T
df_submission.columns = ["sessionID", "TARGET"]
df_submission.to_csv("submission.csv", index=False)

In [64]:
sub_data = pd.read_csv('./submission.csv')
sub_data

,sessionID,TARGET
0,SESSION_252289,22.440000
1,SESSION_252290,1.000000
2,SESSION_252291,2.752580
3,SESSION_252292,3.743524
4,SESSION_252293,7.630333
...,...,...
79781,SESSION_332070,1.000000
79782,SESSION_332071,1.000000
79783,SESSION_332072,2.569325
79784,SESSION_332073,4.449604
